In [62]:
# https://en.wikipedia.org/wiki/List_of_counties_in_California

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib.cm as cm
from matplotlib.collections import PatchCollection
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon
%matplotlib inline

In [2]:
lodes_filepath = 'ca_od_main_JT00_2015.csv'

def load_str(value):
    return str(value)
converters = {'w_geocode' : load_str,
              'h_geocode' : load_str}

# load the lodes data and preserve fips codes as strings bc of preceding zeros
lodes = pd.read_csv(lodes_filepath, converters=converters)

In [4]:
lodes.columns

Index(['w_geocode', 'h_geocode', 'S000', 'SA01', 'SA02', 'SA03', 'SE01',
       'SE02', 'SE03', 'SI01', 'SI02', 'SI03', 'createdate'],
      dtype='object')

In [16]:
lodes['h_geocode'] = lodes['h_geocode'].apply(int)
lodes['w_geocode'] = lodes['w_geocode'].apply(int)

In [17]:
lodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14713690 entries, 0 to 14713689
Data columns (total 13 columns):
w_geocode     int64
h_geocode     int64
S000          int64
SA01          int64
SA02          int64
SA03          int64
SE01          int64
SE02          int64
SE03          int64
SI01          int64
SI02          int64
SI03          int64
createdate    int64
dtypes: int64(13)
memory usage: 1.4 GB


In [39]:
lodes['w_geocode'] = lodes['w_geocode'].apply(int)
lodes['h_geocode'] = lodes['h_geocode'].apply(int)

In [18]:
lodes.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,060014001001007,060014003004007,1,0,1,0,0,0,1,0,0,1,20190826
1,060014001001007,060014027002024,1,1,0,0,0,0,1,0,0,1,20190826
2,060014001001007,060014037011000,1,0,1,0,0,0,1,0,0,1,20190826
3,060014001001007,060014042001011,1,0,1,0,0,0,1,0,0,1,20190826
4,060014001001007,060014042003000,1,0,1,0,0,0,1,0,0,1,20190826


## Determine which census tracts have the highest number of jobs that people commute to from another county.

In [6]:
# load crosswalk table to join by
cxwalk = pd.read_csv("ca_xwalk.csv", encoding="ISO-8859-1", low_memory=False)

In [11]:
cxwalk.head()

,tabblk2010,st,stusps,stname,cty,ctyname,trct,trctname,bgrp,bgrpname,...,stanrcname,necta,nectaname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate
0,60971502032017,6,CA,California,6097,"Sonoma County, CA",6097150203,"1502.03 (Sonoma, CA)",60971502032,"2 (Tract 1502.03, Sonoma, CA)",...,NaN,99999,NaN,NaN,NaN,6000056,56 Sonoma County WIB,38.276402,-122.450695,20190826
1,60971502024011,6,CA,California,6097,"Sonoma County, CA",6097150202,"1502.02 (Sonoma, CA)",60971502024,"4 (Tract 1502.02, Sonoma, CA)",...,NaN,99999,NaN,NaN,NaN,6000056,56 Sonoma County WIB,38.302804,-122.465186,20190826
2,60971502021001,6,CA,California,6097,"Sonoma County, CA",6097150202,"1502.02 (Sonoma, CA)",60971502021,"1 (Tract 1502.02, Sonoma, CA)",...,NaN,99999,NaN,NaN,NaN,6000056,56 Sonoma County WIB,38.308897,-122.446649,20190826
3,60971502023008,6,CA,California,6097,"Sonoma County, CA",6097150202,"1502.02 (Sonoma, CA)",60971502023,"3 (Tract 1502.02, Sonoma, CA)",...,NaN,99999,NaN,NaN,NaN,6000056,56 Sonoma County WIB,38.293432,-122.441744,20190826
4,60971502041022,6,CA,California,6097,"Sonoma County, CA",6097150204,"1502.04 (Sonoma, CA)",60971502041,"1 (Tract 1502.04, Sonoma, CA)",...,NaN,99999,NaN,NaN,NaN,6000056,56 Sonoma County WIB,38.284894,-122.451305,20190826


In [12]:
# sf bay area counties to filter on
cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]

In [13]:
# filter rows in crosswalk to include only SF Bay counties and fields we need
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]
cxwalk = cxwalk[['trct', 'tabblk2010', 'cty', 'ctyname']]

In [18]:
# perform an inner join on crosswalk data and origin destination data to keep only blocks for SF Bay Area
data_w = lodes.merge(cxwalk, how="inner", left_on="w_geocode", right_on="tabblk2010")

In [19]:
# create seperate home and work county fips columns as strings to compare with one another
data_w['cty_h'] = data_w['h_geocode'].apply(str).str.slice(0,4)
data_w['cty_w'] = data_w['w_geocode'].apply(str).str.slice(0,4)
data_w

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
0,60014001001007,60014003004007,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
1,60014001001007,60014027002024,1,1,0,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
2,60014001001007,60014037011000,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
3,60014001001007,60014042001011,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
4,60014001001007,60014042003000,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3432928,60971543043130,60971543043108,1,0,0,1,0,0,1,0,0,1,20190826,6097154304,60971543043130,6097,"Sonoma County, CA",6097,6097
3432929,60971543043130,61150410005040,1,0,1,0,0,1,0,0,0,1,20190826,6097154304,60971543043130,6097,"Sonoma County, CA",6115,6097
3432930,60971543043138,60971530022000,1,0,1,0,0,1,0,0,0,1,20190826,6097154304,60971543043138,6097,"Sonoma County, CA",6097,6097
3432931,60971543043138,60971537043012,1,0,1,0,1,0,0,0,0,1,20190826,6097154304,60971543043138,6097,"Sonoma County, CA",6097,6097


In [20]:
# filter od blocks to those that have different work and home county fips
data_w_filtered = data_w[(data_w['cty_h'] != data_w['cty_w'])]

In [25]:
# inspect data
data_w_filtered.head(10)

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
0,60014001001007,060014003004007,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
1,60014001001007,060014027002024,1,1,0,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
2,60014001001007,060014037011000,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
3,60014001001007,060014042001011,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
4,60014001001007,060014042003000,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
5,60014001001007,060014042003004,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
6,60014001001007,060014045023003,1,0,0,1,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
7,60014001001007,060014046001000,1,0,0,1,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
8,60014001001007,060014046001009,1,0,0,1,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001
9,60014001001007,060014049003007,1,0,1,0,0,0,1,0,0,1,20190826,6001400100,60014001001007,6001,"Alameda County, CA",0600,6001


In [21]:
# aggregate blocks to tracts and sum S000 column
data_agg = data_w_filtered.groupby(['trct', 'cty'], as_index=False)[['S000']].sum()

In [22]:
# sort by S000 column descending
data_agg = data_agg.sort_values('S000', ascending=False)

In [23]:
# inspect the data
data_agg.head(10)

,trct,cty,S000
850,6075061500,6075,83356
674,6075011700,6075,67945
270,6001441503,6001,42289
884,6081602300,6081,24844
1175,6085505006,6085,24833
725,6075017601,6075,24001
1319,6085511705,6085,20124
726,6075017700,6075,19461
665,6075010500,6075,17851
1167,6085504602,6085,15550


In [32]:
dirname = os.path.dirname(os.path.realpath("__file__"))
dirname

'C:\\Users\\SIDDHARTH\\OneDrive\\Documents'

In [33]:
# write processed data to csv file
import sys, os
outfile = os.path.join(dirname, 'lehd_od_tracts_high_migration.csv')
data_agg.to_csv(outfile, columns=['trct', 'cty', 'S000'], index=False, encoding="utf-8")

## Determine which census tracts have the highest number of jobs that people commute to from another county including greater bay area.

In [34]:
# greater bay area counties
cty_fips_list = [6001, 6013, 6041, 6047, 6055, 6067, 6069, 6075, 6077, 6081, 6085, 6087, 6095, 6097, 6099]

In [35]:
# sf bay area counties
target_cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]

In [36]:
# filter rows in crosswalk to include only SF Bay counties and fields we need
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]
cxwalk = cxwalk[['trct', 'tabblk2010', 'cty', 'ctyname']]

In [40]:
data_h = lodes.merge(cxwalk, how="inner", left_on="h_geocode", right_on="tabblk2010")

In [41]:
# create seperate home and work county fips columns as strings to compare with one another
data_h['cty_h'] = data_h['h_geocode'].apply(str).str.slice(0,4)
data_h['cty_w'] = data_h['w_geocode'].apply(str).str.slice(0,4)
data_h['cty_h'] = pd.to_numeric(data_h['cty_h'], errors='coerce')
data_h['cty_w'] = pd.to_numeric(data_h['cty_w'], errors='coerce')

In [42]:
# inspect data
data_h.head(10)
# data_h.dtypes

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
0,60014001001007,60014003004007,1,0,1,0,0,0,1,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
1,60014003003017,60014003004007,1,0,0,1,0,1,0,0,1,0,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
2,60014003003018,60014003004007,1,1,0,0,0,1,0,1,0,0,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
3,60014003004011,60014003004007,1,0,1,0,1,0,0,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
4,60014010001002,60014003004007,1,1,0,0,0,0,1,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
5,60014011004018,60014003004007,1,1,0,0,1,0,0,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
6,60014028001007,60014003004007,1,1,0,0,0,1,0,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
7,60014028002035,60014003004007,1,0,0,1,0,0,1,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
8,60014030001002,60014003004007,1,0,0,1,0,0,1,0,1,0,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
9,60014033001008,60014003004007,1,0,1,0,0,1,0,0,0,1,20190826,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001


In [43]:
# aggregate blocks to tracts while computing the number of local and away jobs per tract
data_agg = data_h.groupby(['trct', 'cty'], as_index=False).apply(lambda x: pd.Series(dict(
    sum_local=(x.cty_h == x.cty_w).sum(),
    sum_away=((x.cty_h != x.cty_w) & x.cty_w.isin(target_cty_fips_list)).sum() # only counting away as different from home county and within the 9 county bay area
)))

In [44]:
# compute proportion of away vs local
data_agg['ratio_away_to_local'] = data_agg['sum_away'] / data_agg['sum_local']

In [45]:
# sort by ratio column descending
data_agg = data_agg.sort_values('ratio_away_to_local', ascending=False)

In [46]:
# inspect the data
data_agg

,,sum_local,sum_away,ratio_away_to_local
trct,cty,,,
6095253000,6095,27,634,23.481481
6041122000,6041,21,388,18.476190
6041990100,6041,12,141,11.750000
6055200900,6055,23,193,8.391304
6081990100,6081,23,137,5.956522
...,...,...,...,...
6085512312,6085,1064,231,0.217105
6085502907,6085,1536,327,0.212891
6085512311,6085,1241,263,0.211926


In [47]:
# write processed data to csv file
outfile = os.path.join(dirname, 'lehd_od_tracts_home_ratio_gba.csv')
data_agg.to_csv(outfile, encoding="utf-8")

# Now we shall determine which tracts have the highest number of jobs that people commute daily 

In [48]:
# aggregate blocks to tracts and sum S000 column
data_agg = data_h.groupby(['trct', 'cty'], as_index=False).apply(lambda x: pd.Series(dict(
    sum_local=(x.cty_h == x.cty_w).sum(),
    sum_away=(x.cty_h != x.cty_w).sum()
)))

In [50]:
# compute proportion of away vs local
data_agg['ratio_away_to_local'] = data_agg['sum_away'] / data_agg['sum_local']

In [51]:
# sort by ratio column descending
data_agg = data_agg.sort_values('ratio_away_to_local', ascending=False)

In [53]:
data_agg.head(10)

,,sum_local,sum_away,ratio_away_to_local
trct,cty,,,
6095253000,6095,27,1096,40.592593
6041122000,6041,21,560,26.666667
6041990100,6041,12,141,11.750000
6055200900,6055,23,270,11.739130
6095252802,6095,42,320,7.619048
6081990100,6081,23,138,6.000000
6013389100,6013,154,914,5.935065
6013391000,6013,173,944,5.456647
6013389200,6013,143,746,5.216783


In [60]:
# write processed data to csv file
outfile = os.path.join(dirname, 'lehd_od_tracts_home_ratio.csv')
data_agg.to_csv(outfile, encoding="utf-8")

In [58]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [57]:
lodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14713690 entries, 0 to 14713689
Data columns (total 13 columns):
w_geocode     int64
h_geocode     int64
S000          int64
SA01          int64
SA02          int64
SA03          int64
SE01          int64
SE02          int64
SE03          int64
SI01          int64
SI02          int64
SI03          int64
createdate    int64
dtypes: int64(13)
memory usage: 1.4 GB


In [4]:
lodes.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,060014001001007,060014003004007,1,0,1,0,0,0,1,0,0,1,20190826
1,060014001001007,060014027002024,1,1,0,0,0,0,1,0,0,1,20190826
2,060014001001007,060014037011000,1,0,1,0,0,0,1,0,0,1,20190826
3,060014001001007,060014042001011,1,0,1,0,0,0,1,0,0,1,20190826
4,060014001001007,060014042003000,1,0,1,0,0,0,1,0,0,1,20190826


In [5]:
# extract home and work tract fips codes from the block fips codes
lodes['h_tract'] = lodes['h_geocode'].map(lambda x: x[0:11])
lodes['w_tract'] = lodes['w_geocode'].map(lambda x: x[0:11])

In [20]:
# how many workers commute from each tract
leave_from = lodes.groupby('h_tract')['S000'].sum()

In [21]:
# how many workers commute to each tract
travel_to = lodes.groupby('w_tract')['S000'].sum()

In [22]:
# verify that the counts match up
leave_from.sum() == travel_to.sum()

True

In [23]:
# create df indexed by tract with columns for number of workers leaving from and commuting to
commutes = pd.DataFrame(data={'travel_to':travel_to, 'leave_from':leave_from})
len(commutes)

8057

In [6]:
commutes.head()

NameError: name 'commutes' is not defined